Installing Necessary Resources

In [ ]:
# Install required packages
pip install facenet-pytorch mtcnn opencv-python

Trial 4 - (Best One among all till now)

In [2]:
import cv2
import os
import torch
import pickle
import json
import numpy as np
from datetime import datetime, timedelta
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk, Tk, Label, Button, Entry, StringVar, Toplevel, Frame, messagebox, scrolledtext
from tkinter import filedialog
from facenet_pytorch import MTCNN, InceptionResnetV1

# Configuration
os.makedirs("faces", exist_ok=True)
os.makedirs("attendance", exist_ok=True)
EMBEDDING_PATH = "face_embeddings.pkl"
ATTENDANCE_INTERVAL = 1  # minutes
ATTENDANCE_FILE = "attendance/attendance_records.json"

# Initialize models
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=160, margin=20, min_face_size=40, device=device, keep_all=True)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

class AttendanceSystem:
# Add this method to the FaceAttendanceApp class
    

# Add this button to the control panel in __init__ (with the other buttons)

    def __init__(self):
        self.attendance_records = []
        self.current_session = {}
        self.last_attendance_time = None
        self.attendance_active = False
        self.load_attendance()
    
    def load_attendance(self):
        if os.path.exists(ATTENDANCE_FILE):
            with open(ATTENDANCE_FILE, 'r') as f:
                self.attendance_records = json.load(f)
    
    def save_attendance(self):
        with open(ATTENDANCE_FILE, 'w') as f:
            json.dump(self.attendance_records, f, indent=4)
    
    def mark_present(self, name):
        now = datetime.now()
        time_str = now.strftime("%Y-%m-%d %H:%M:%S")
        
        if name not in self.current_session:
            self.current_session[name] = time_str
            return True
        return False
    
    def check_attendance_time(self):
        if not self.attendance_active or self.last_attendance_time is None:
            return False
            
        now = datetime.now()
        if (now - self.last_attendance_time) >= timedelta(minutes=ATTENDANCE_INTERVAL):
            self.finalize_attendance()
            self.last_attendance_time = None
            self.attendance_active = False
            return True
        return False
    
    def start_attendance_session(self):
        self.last_attendance_time = datetime.now()
        self.attendance_active = True
        self.current_session = {}
    
    def finalize_attendance(self):
        if not self.current_session:
            return
            
        db = load_embeddings()
        all_names = set(db.keys())
        present_names = set(self.current_session.keys())
        absent_names = all_names - present_names
        
        session_data = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "present": {name: self.current_session[name] for name in present_names},
            "absent": list(absent_names)
        }
        
        self.attendance_records.append(session_data)
        self.save_attendance()
        self.current_session = {}
        return session_data
    
    #NewFunction
    # Add this method to the FaceAttendanceApp class


#NewFunction ended

def load_embeddings():
    if os.path.exists(EMBEDDING_PATH):
        with open(EMBEDDING_PATH, 'rb') as f:
            return pickle.load(f)
    return {}

def save_embeddings(data):
    with open(EMBEDDING_PATH, 'wb') as f:
        pickle.dump(data, f)

class FaceAttendanceApp:
    def __init__(self, master):
        self.master = master
        self.attendance_system = AttendanceSystem()
        
        # Window configuration
        master.title("Smart Attendance System")
        master.geometry("1000x700")
        master.configure(bg='#f5f5f5')
        
        # Custom style
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Color scheme
        self.colors = {
            'primary': '#4e73df',
            'success': '#1cc88a',
            'info': '#36b9cc',
            'warning': '#f6c23e',
            'danger': '#e74a3b',
            'light': '#f8f9fc',
            'dark': '#5a5c69'
        }
        
        # Configure styles
        self.configure_styles()
        
        # Header frame
        header_frame = ttk.Frame(master, style='Header.TFrame')
        header_frame.pack(fill='x', padx=10, pady=10)
        
        ttk.Label(header_frame, text="Smart Attendance System", 
                 style='Header.TLabel').pack(pady=15)
        
        # Main content frame
        main_frame = ttk.Frame(master)
        main_frame.pack(fill='both', expand=True, padx=20, pady=10)
        
        # Left panel - Controls
        control_frame = ttk.Frame(main_frame, style='Card.TFrame')
        control_frame.pack(side='left', fill='y', padx=(0, 10))
        
        ttk.Label(control_frame, text="System Controls", 
                 style='CardHeader.TLabel').pack(pady=10)
        
        control_buttons = [
            ("Add New Student", self.add_face_gui, 'primary'),
            ("Start Attendance", self.recognize_faces, 'success'),
            ("Manage Students", self.manage_users, 'info'),
            ("View Attendance", self.view_attendance, 'warning'),
            ("Edit Attendance", self.edit_attendance, 'primary')  # New button
        ]
        
        for text, command, color in control_buttons:
            btn = ttk.Button(control_frame, text=text, command=command,
                            style=f'{color}.TButton')
            btn.pack(fill='x', padx=10, pady=5)
        
        # Right panel - Status
            status_frame = ttk.Frame(main_frame, style='Card.TFrame')
            status_frame.pack(side='right', fill='both', expand=True)
        
            ttk.Label(status_frame, text="Attendance Status", 
                 style='CardHeader.TLabel').pack(pady=10)
        
            self.status_text = scrolledtext.ScrolledText(status_frame, 
                                                   height=20,
                                                   font=('Helvetica', 10),
                                                   wrap=tk.WORD)
            self.status_text.pack(fill='both', expand=True, padx=10, pady=5)
        
        # Footer
            footer_frame = ttk.Frame(master, style='Footer.TFrame')
            footer_frame.pack(fill='x', padx=10, pady=10)
        
            self.time_label = ttk.Label(footer_frame, 
                 text="Attendance not started",
                 style='Footer.TLabel')
            self.time_label.pack()
    
    def edit_attendance(self):
    # Create a new window for manual attendance editing
        edit_window = Toplevel(self.master)
        edit_window.title("Edit Attendance")
        edit_window.geometry("800x600")
    
    # Main frame
        main_frame = ttk.Frame(edit_window)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
    
    # Header
        ttk.Label(main_frame, text="Edit Attendance Records", 
             style='Header.TLabel').pack(pady=10)
    
    # Date selection
        date_frame = ttk.Frame(main_frame)
        date_frame.pack(fill='x', pady=10)
    
        ttk.Label(date_frame, text="Select Session:").pack(side='left')
    
    # Load available sessions
        if not os.path.exists(ATTENDANCE_FILE):
            messagebox.showerror("Error", "No attendance records found")
            edit_window.destroy()
            return
    
        with open(ATTENDANCE_FILE, 'r') as f:
            records = json.load(f)
    
        session_dates = [r['timestamp'] for r in records]
        session_var = StringVar()
        session_dropdown = ttk.Combobox(date_frame, textvariable=session_var, 
                                   values=session_dates, state='readonly')
        session_dropdown.pack(side='left', padx=10)
        session_dropdown.current(0)
    
    # Load button
        def load_session():
            selected_session = session_var.get()
            if not selected_session:
                return
            
        # Find the selected session
            session_data = next((r for r in records if r['timestamp'] == selected_session), None)
            if not session_data:
                return
            
        # Clear previous entries
            for widget in student_frame.winfo_children():
                widget.destroy()
            
        # Get all registered students
            db = load_embeddings()
            all_students = list(db.keys())
        
        # Create attendance list
            for student in all_students:
                row_frame = ttk.Frame(student_frame)
                row_frame.pack(fill='x', pady=2)
            
            # Student name
                ttk.Label(row_frame, text=student, width=20).pack(side='left')
            
            # Attendance status
                status_var = StringVar()
                status_var.set('present' if student in session_data['present'] else 'absent')
            
                ttk.Radiobutton(row_frame, text="Present", variable=status_var, 
                           value='present').pack(side='left', padx=5)
                ttk.Radiobutton(row_frame, text="Absent", variable=status_var, 
                           value='absent').pack(side='left', padx=5)
            
            # Store reference to update later
                student_status_vars[student] = status_var
    
        ttk.Button(date_frame, text="Load Session", command=load_session, 
              style='primary.TButton').pack(side='left')
    
    # Student list frame with scrollbar
        student_frame = ttk.Frame(main_frame)
        student_frame.pack(fill='both', expand=True)
    
    # Container for student status variables
        student_status_vars = {}
    
    # Save button
        def save_changes():
            selected_session = session_var.get()
            if not selected_session:
                return
            
        # Find the session to update
            session_index = next((i for i, r in enumerate(records) 
                            if r['timestamp'] == selected_session), None)
            if session_index is None:
                return
            
        # Update present/absent lists based on current selections
            present = {}
            absent = []
        
            for student, var in student_status_vars.items():
                if var.get() == 'present':
                # Try to preserve original timestamp if available
                    original_time = records[session_index]['present'].get(student, 
                        datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    present[student] = original_time
                else:
                    absent.append(student)
        
        # Update the record
            records[session_index]['present'] = present
            records[session_index]['absent'] = absent
        
        # Save back to file
            with open(ATTENDANCE_FILE, 'w') as f:
                json.dump(records, f, indent=4)

            messagebox.showinfo("Success", "Attendance records updated successfully")
    
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=10)
    
        ttk.Button(button_frame, text="Save Changes", command=save_changes,
              style='success.TButton').pack(side='left', padx=5)
        ttk.Button(button_frame, text="Cancel", command=edit_window.destroy,
              style='danger.TButton').pack(side='left', padx=5)
    
    # Load the first session by default
        load_session()
          
    def configure_styles(self):
        self.style.configure('Header.TFrame', background=self.colors['primary'])
        self.style.configure('Header.TLabel', 
                           background=self.colors['primary'],
                           foreground='white',
                           font=('Helvetica', 18, 'bold'))
        
        self.style.configure('Card.TFrame', 
                           background='white',
                           relief='groove',
                           borderwidth=2)
        
        self.style.configure('CardHeader.TLabel',
                           font=('Helvetica', 12, 'bold'),
                           background='white',
                           foreground=self.colors['dark'])
        
        self.style.configure('Footer.TFrame',
                           background=self.colors['light'])
        
        self.style.configure('Footer.TLabel',
                           background=self.colors['light'],
                           font=('Helvetica', 10))
        
        for color_name, color_code in self.colors.items():
            if color_name in ['primary', 'success', 'info', 'warning', 'danger']:
                self.style.configure(f'{color_name}.TButton',
                                    foreground='white',
                                    background=color_code,
                                    font=('Helvetica', 10, 'bold'),
                                    padding=8)
                
                self.style.map(f'{color_name}.TButton',
                              background=[('active', color_code), ('disabled', '#dddddd')])
    
    def update_status(self, message):
        self.status_text.insert(tk.END, message + "\n")
        self.status_text.see(tk.END)
        self.status_text.update()
    
    def add_face_gui(self):
        def start_capture():
            name = name_var.get().strip()
            if not name:
                messagebox.showerror("Error", "Please enter a valid name")
                return
            
            cap = cv2.VideoCapture(0)
            top.destroy()
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    continue
                
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(rgb_frame)
                boxes, _ = mtcnn.detect(img)
                
                if boxes is not None:
                    for box in boxes:
                        x1, y1, x2, y2 = map(int, box)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                cv2.imshow("Capture Face - Press 'C' to Capture", frame)
                key = cv2.waitKey(1) & 0xFF
                
                if key == ord('c'):
                    faces = mtcnn(img)
                    if faces is not None:
                        embedding = resnet(faces[0].unsqueeze(0).to(device)).detach().cpu()
                        db = load_embeddings()
                        db[name] = embedding
                        save_embeddings(db)
                        
                        img_path = os.path.join("faces", f"{name}.jpg")
                        cv2.imwrite(img_path, frame)
                        messagebox.showinfo("Success", f"{name} registered successfully!")
                    else:
                        messagebox.showerror("Error", "No face detected")
                    break
                elif key == ord('q'):
                    break
            
            cap.release()
            cv2.destroyAllWindows()
        
        top = Toplevel(self.master)
        top.title("Register New Student")
        top.geometry("350x200")
        top.configure(bg='white')
        
        ttk.Label(top, text="Enter Student Name:", 
                 style='CardHeader.TLabel').pack(pady=10)
        
        name_var = StringVar()
        name_entry = ttk.Entry(top, textvariable=name_var, font=('Helvetica', 12))
        name_entry.pack(pady=5, padx=20, fill='x')
        
        btn_frame = ttk.Frame(top)
        btn_frame.pack(pady=10)
        
        ttk.Button(btn_frame, text="Start Capture", 
                  command=start_capture, style='primary.TButton').pack(side='left', padx=5)
        ttk.Button(btn_frame, text="Cancel", 
                  command=top.destroy, style='danger.TButton').pack(side='left', padx=5)
    
    def recognize_faces(self):
        # Start the attendance session
        self.attendance_system.start_attendance_session()
        self.update_status("\nAttendance session started...")
        
        # Start updating the timer display
        self.update_timer()
        
        db = load_embeddings()
        if not db:
            messagebox.showinfo("Info", "No registered students found")
            return
        
        names = list(db.keys())
        embeddings = torch.stack(list(db.values())).squeeze(1)
        
        cap = cv2.VideoCapture(0)
        self.update_status("Starting face recognition...")
        
        while True:
            # Check if we should stop (attendance period over)
            if not self.attendance_system.attendance_active:
                self.update_status("Attendance period completed. Session closed.")
                break
                
            ret, frame = cap.read()
            if not ret:
                continue
            
            # Check if attendance interval has passed
            if self.attendance_system.check_attendance_time():
                session_data = self.attendance_system.finalize_attendance()
                self.update_status(f"Attendance marked at {datetime.now().strftime('%H:%M:%S')}")
                self.update_status(f"Present: {len(session_data['present'])}")
                self.update_status(f"Absent: {len(session_data['absent'])}")
                break
            
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(rgb_frame)
            boxes, _ = mtcnn.detect(img)
            
            if boxes is not None:
                faces = mtcnn(img)
                if faces is not None:
                    current_embeddings = resnet(faces.to(device)).detach().cpu()
                    
                    for i, (box, emb) in enumerate(zip(boxes, current_embeddings)):
                        x1, y1, x2, y2 = map(int, box)
                        dists = torch.norm(embeddings - emb, dim=1)
                        min_dist, idx = torch.min(dists, dim=0)
                        
                        if min_dist < 0.7:
                            name = names[idx]
                            is_new = self.attendance_system.mark_present(name)
                            
                            if is_new:
                                self.update_status(f"{name} marked present at {datetime.now().strftime('%H:%M:%S')}")
                                label = f"{name} (Present)"
                                color = (0, 120, 255)  # Orange
                            else:
                                label = f"{name} (Already marked)"
                                color = (0, 180, 0)  # Dark green
                        else:
                            label = "Unknown"
                            color = (0, 0, 255)  # Red
                        
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(frame, label, (x1, y1-10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # Display time until next attendance marking
            if self.attendance_system.last_attendance_time:
                time_left = (self.attendance_system.last_attendance_time + 
                            timedelta(minutes=ATTENDANCE_INTERVAL) - datetime.now())
                mins, secs = divmod(max(0, time_left.seconds), 60)
                time_text = f"Time left: {mins:02d}:{secs:02d}"
                
                cv2.rectangle(frame, (5, 5), (250, 40), (0, 0, 0), -1)
                cv2.putText(frame, time_text, (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.imshow("Face Recognition - Press Q to Quit", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.attendance_system.attendance_active = False
                break
        
        cap.release()
        cv2.destroyAllWindows()
    
    def update_timer(self):
        if self.attendance_system.attendance_active and self.attendance_system.last_attendance_time:
            time_left = (self.attendance_system.last_attendance_time + 
                        timedelta(minutes=ATTENDANCE_INTERVAL) - datetime.now())
            if time_left.total_seconds() > 0:
                mins, secs = divmod(time_left.seconds, 60)
                self.time_label.config(text=f"Time left: {mins:02d}:{secs:02d}")
                self.master.after(1000, self.update_timer)
            else:
                self.time_label.config(text="Attendance session completed")
        else:
            self.time_label.config(text="Attendance not started")
    
    def manage_users(self):
        top = Toplevel(self.master)
        top.title("Student Management")
        top.geometry("900x600")
        top.configure(bg='white')
        
        main_frame = ttk.Frame(top)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Registered Students", 
                 style='Header.TLabel').pack(pady=10)
        
        # Create a canvas and scrollbar
        canvas = tk.Canvas(main_frame, bg='white')
        scrollbar = ttk.Scrollbar(main_frame, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Load registered users
        db = load_embeddings()
        users = list(db.keys())
        
        if not users:
            ttk.Label(scrollable_frame, text="No registered students").pack()
            return
        
        # Create a grid of student cards
        row_frame = None
        for i, user in enumerate(users):
            if i % 4 == 0:
                row_frame = ttk.Frame(scrollable_frame)
                row_frame.pack(fill='x', pady=5)
            
            card_frame = ttk.Frame(row_frame, style='Card.TFrame')
            card_frame.pack(side='left', padx=10, pady=5)
            
            # Student image
            img_path = os.path.join("faces", f"{user}.jpg")
            if os.path.exists(img_path):
                img = Image.open(img_path).resize((120, 120))
                photo = ImageTk.PhotoImage(img)
                lbl = ttk.Label(card_frame, image=photo)
                lbl.image = photo
                lbl.pack(pady=5)
            
            # Student name
            ttk.Label(card_frame, text=user, 
                     style='CardHeader.TLabel').pack()
            
            # Delete button
            ttk.Button(card_frame, text="Delete", 
                      command=lambda u=user: self.delete_user(u, top),
                      style='danger.TButton').pack(pady=5)
    
    def delete_user(self, user, window):
        if not messagebox.askyesno("Confirm", f"Delete {user} permanently?"):
            return
        
        db = load_embeddings()
        if user in db:
            del db[user]
            save_embeddings(db)
            
            img_path = os.path.join("faces", f"{user}.jpg")
            if os.path.exists(img_path):
                os.remove(img_path)
            
            messagebox.showinfo("Success", f"Student {user} deleted successfully")
            window.destroy()
            self.manage_users()
        else:
            messagebox.showerror("Error", "Student not found")
    
    def view_attendance(self):
        top = Toplevel(self.master)
        top.title("Attendance Records")
        top.geometry("1000x600")
        
        main_frame = ttk.Frame(top)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Attendance History", 
                 style='Header.TLabel').pack(pady=10)
        
        # Create a text widget with scrollbar
        text_frame = ttk.Frame(main_frame)
        text_frame.pack(fill='both', expand=True)
        
        scrollbar = ttk.Scrollbar(text_frame)
        scrollbar.pack(side='right', fill='y')
        
        attendance_text = tk.Text(text_frame, wrap=tk.WORD, 
                                yscrollcommand=scrollbar.set,
                                font=('Courier New', 10))
        attendance_text.pack(fill='both', expand=True)
        
        scrollbar.config(command=attendance_text.yview)
        
        # Load and display attendance records
        if not os.path.exists(ATTENDANCE_FILE):
            attendance_text.insert(tk.END, "No attendance records found")
            return
        
        with open(ATTENDANCE_FILE, 'r') as f:
            records = json.load(f)
        
        for record in records:
            timestamp = record['timestamp']
            present = record['present']
            absent = record['absent']
            
            attendance_text.insert(tk.END, f"\n=== Session: {timestamp} ===\n")
            attendance_text.insert(tk.END, f"Present ({len(present)}):\n")
            for name, time in present.items():
                attendance_text.insert(tk.END, f"  - {name} at {time}\n")
            
            attendance_text.insert(tk.END, f"\nAbsent ({len(absent)}):\n")
            for name in absent:
                attendance_text.insert(tk.END, f"  - {name}\n")
            
            attendance_text.insert(tk.END, "\n" + "="*40 + "\n")
        
        attendance_text.config(state='disabled')

if __name__ == "__main__":
    root = Tk()
    app = FaceAttendanceApp(root)
    root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Python\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\anany\AppData\Local\Temp\ipykernel_4048\1387209220.py", line 462, in recognize_faces
    self.update_status(f"Present: {len(session_data['present'])}")
TypeError: 'NoneType' object is not subscriptable


Trail 6: Video is recorded for embeddings

In [ ]:
import cv2
import os
import torch
import pickle
import json
import numpy as np
from datetime import datetime, timedelta
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk, Tk, Label, Button, Entry, StringVar, Toplevel, Frame, messagebox, scrolledtext
from tkinter import filedialog
from facenet_pytorch import MTCNN, InceptionResnetV1
import time
from threading import Thread

# Configuration
os.makedirs("faces", exist_ok=True)
os.makedirs("attendance", exist_ok=True)
EMBEDDING_PATH = "face_embeddings.pkl"
ATTENDANCE_FILE = "attendance/attendance_records.json"
ATTENDANCE_INTERVAL = 1  # minutes
VIDEO_LENGTH = 15  # Increased from 10 to 15 seconds for more embeddings
MIN_EMBEDDINGS = 30  # Increased minimum embeddings to capture

# Initialize models
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=160, margin=20, min_face_size=40, device=device, keep_all=True)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def load_embeddings():
    if os.path.exists(EMBEDDING_PATH):
        with open(EMBEDDING_PATH, 'rb') as f:
            return pickle.load(f)
    return {}

def save_embeddings(data):
    with open(EMBEDDING_PATH, 'wb') as f:
        pickle.dump(data, f)

class AttendanceSystem:
    def __init__(self):
        self.attendance_records = []
        self.current_session = {}
        self.last_attendance_time = None
        self.attendance_active = False
        self.load_attendance()
    
    def load_attendance(self):
        if os.path.exists(ATTENDANCE_FILE):
            with open(ATTENDANCE_FILE, 'r') as f:
                self.attendance_records = json.load(f)
    
    def save_attendance(self):
        with open(ATTENDANCE_FILE, 'w') as f:
            json.dump(self.attendance_records, f, indent=4)
    
    def mark_present(self, name):
        now = datetime.now()
        time_str = now.strftime("%Y-%m-%d %H:%M:%S")
        
        if name not in self.current_session:
            self.current_session[name] = time_str
            return True
        return False
    
    def check_attendance_time(self):
        if not self.attendance_active or self.last_attendance_time is None:
            return False
            
        now = datetime.now()
        if (now - self.last_attendance_time) >= timedelta(minutes=ATTENDANCE_INTERVAL):
            self.finalize_attendance()
            self.last_attendance_time = None
            self.attendance_active = False
            return True
        return False
    
    def start_attendance_session(self):
        self.last_attendance_time = datetime.now()
        self.attendance_active = True
        self.current_session = {}
    
    def finalize_attendance(self):
        if not self.current_session:
            return
            
        db = load_embeddings()
        all_names = set(db.keys())
        present_names = set(self.current_session.keys())
        absent_names = all_names - present_names
        
        session_data = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "present": {name: self.current_session[name] for name in present_names},
            "absent": list(absent_names)
        }
        
        self.attendance_records.append(session_data)
        self.save_attendance()
        self.current_session = {}
        return session_data

class FaceAttendanceApp:
    def __init__(self, master):
        self.master = master
        self.attendance_system = AttendanceSystem()
        self.cap = None  # VideoCapture object
        
        # Window configuration
        master.title("Smart Attendance System")
        master.geometry("1000x700")
        master.configure(bg='#f5f5f5')
        
        # Custom style
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Color scheme
        self.colors = {
            'primary': '#4e73df',
            'success': '#1cc88a',
            'info': '#36b9cc',
            'warning': '#f6c23e',
            'danger': '#e74a3b',
            'light': '#f8f9fc',
            'dark': '#5a5c69'
        }
        
        # Configure styles
        self.configure_styles()
        
        # Header frame
        header_frame = ttk.Frame(master, style='Header.TFrame')
        header_frame.pack(fill='x', padx=10, pady=10)
        
        ttk.Label(header_frame, text="Smart Attendance System", 
                 style='Header.TLabel').pack(pady=15)
        
        # Main content frame
        main_frame = ttk.Frame(master)
        main_frame.pack(fill='both', expand=True, padx=20, pady=10)
        
        # Left panel - Controls
        control_frame = ttk.Frame(main_frame, style='Card.TFrame')
        control_frame.pack(side='left', fill='y', padx=(0, 10))
        
        ttk.Label(control_frame, text="System Controls", 
                 style='CardHeader.TLabel').pack(pady=10)
        
        control_buttons = [
            ("Add New Student", self.add_face_gui, 'primary'),
            ("Start Attendance", self.recognize_faces, 'success'),
            ("Manage Students", self.manage_users, 'info'),
            ("View Attendance", self.view_attendance, 'warning'),
            ("Edit Attendance", self.edit_attendance, 'primary')
        ]
        
        for text, command, color in control_buttons:
            btn = ttk.Button(control_frame, text=text, command=command,
                            style=f'{color}.TButton')
            btn.pack(fill='x', padx=10, pady=5)
        
        # Right panel - Status
        status_frame = ttk.Frame(main_frame, style='Card.TFrame')
        status_frame.pack(side='right', fill='both', expand=True)
        
        ttk.Label(status_frame, text="Attendance Status", 
                 style='CardHeader.TLabel').pack(pady=10)
        
        self.status_text = scrolledtext.ScrolledText(status_frame, 
                                                   height=20,
                                                   font=('Helvetica', 10),
                                                   wrap=tk.WORD)
        self.status_text.pack(fill='both', expand=True, padx=10, pady=5)
        
        # Footer
        footer_frame = ttk.Frame(master, style='Footer.TFrame')
        footer_frame.pack(fill='x', padx=10, pady=10)
        
        self.time_label = ttk.Label(footer_frame, 
                 text="Attendance not started",
                 style='Footer.TLabel')
        self.time_label.pack()
    
    def configure_styles(self):
        self.style.configure('Header.TFrame', background=self.colors['primary'])
        self.style.configure('Header.TLabel', 
                           background=self.colors['primary'],
                           foreground='white',
                           font=('Helvetica', 18, 'bold'))
        
        self.style.configure('Card.TFrame', 
                           background='white',
                           relief='groove',
                           borderwidth=2)
        
        self.style.configure('CardHeader.TLabel',
                           font=('Helvetica', 12, 'bold'),
                           background='white',
                           foreground=self.colors['dark'])
        
        self.style.configure('Footer.TFrame',
                           background=self.colors['light'])
        
        self.style.configure('Footer.TLabel',
                           background=self.colors['light'],
                           font=('Helvetica', 10))
        
        for color_name, color_code in self.colors.items():
            if color_name in ['primary', 'success', 'info', 'warning', 'danger']:
                self.style.configure(f'{color_name}.TButton',
                                    foreground='white',
                                    background=color_code,
                                    font=('Helvetica', 10, 'bold'),
                                    padding=8)
                
                self.style.map(f'{color_name}.TButton',
                              background=[('active', color_code), ('disabled', '#dddddd')])
    
    def update_status(self, message):
        self.status_text.insert(tk.END, message + "\n")
        self.status_text.see(tk.END)
        self.status_text.update()
    
    def add_face_gui(self):
        def start_video_capture():
            name = name_var.get().strip()
            if not name:
                messagebox.showerror("Error", "Please enter a valid name")
                return
            
            top.destroy()
            self.capture_video_embeddings(name)
        
        top = Toplevel(self.master)
        top.title("Register New Student")
        top.geometry("350x200")
        top.configure(bg='white')
        
        ttk.Label(top, text="Enter Student Name:", 
                 style='CardHeader.TLabel').pack(pady=10)
        
        name_var = StringVar()
        name_entry = ttk.Entry(top, textvariable=name_var, font=('Helvetica', 12))
        name_entry.pack(pady=5, padx=20, fill='x')
        
        btn_frame = ttk.Frame(top)
        btn_frame.pack(pady=10)
        
        ttk.Button(btn_frame, text="Record Video", 
                  command=start_video_capture, style='primary.TButton').pack(side='left', padx=5)
        ttk.Button(btn_frame, text="Cancel", 
                  command=top.destroy, style='danger.TButton').pack(side='left', padx=5)
    
    def capture_video_embeddings(self, name):
        """Capture video and extract multiple face embeddings"""
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Could not open video device")
            return
            
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        video_path = os.path.join("faces", f"{name}.avi")
        out = cv2.VideoWriter(video_path, fourcc, 20.0, (640, 480))
        
        embeddings = []
        start_time = time.time()
        capture_complete = False
        
        # Create a progress window
        progress_window = Toplevel(self.master)
        progress_window.title("Recording Video")
        progress_window.geometry("400x150")
        
        progress_label = ttk.Label(progress_window, text="Please move your head slowly in different directions")
        progress_label.pack(pady=10)
        
        progress_var = tk.DoubleVar()
        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, 
                                      maximum=VIDEO_LENGTH, length=350)
        progress_bar.pack(pady=10)
        
        time_label = ttk.Label(progress_window, text=f"Time remaining: {VIDEO_LENGTH} seconds")
        time_label.pack()
        
        def update_progress():
            while not capture_complete:
                elapsed = time.time() - start_time
                remaining = max(0, VIDEO_LENGTH - elapsed)
                progress_var.set(elapsed)
                time_label.config(text=f"Time remaining: {int(remaining)} seconds")
                if elapsed >= VIDEO_LENGTH:
                    break
                time.sleep(0.1)
        
        # Start progress update thread
        progress_thread = Thread(target=update_progress)
        progress_thread.start()
        
        # Main capture loop
        while (time.time() - start_time) < VIDEO_LENGTH:
            ret, frame = self.cap.read()
            if not ret:
                continue
            
            # Write frame to video file
            out.write(frame)
            
            # Process frame for face detection
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(rgb_frame)
            
            # Detect faces
            boxes, _ = mtcnn.detect(img)
            
            if boxes is not None:
                faces = mtcnn(img)
                if faces is not None:
                    # Get embeddings for all detected faces
                    current_embeddings = resnet(faces.to(device)).detach().cpu()
                    embeddings.extend(current_embeddings)
            
            # Display instructions and count
            cv2.putText(frame, "Move your head slowly in different directions", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Embeddings captured: {len(embeddings)}/{MIN_EMBEDDINGS}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Time remaining: {int(VIDEO_LENGTH - (time.time() - start_time))}s", 
                        (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.imshow("Recording Video - Move Your Head", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            # Early exit if we have enough embeddings
            if len(embeddings) >= MIN_EMBEDDINGS:
                break
        
        # Clean up
        capture_complete = True
        self.cap.release()
        out.release()
        cv2.destroyAllWindows()
        progress_window.destroy()
        
        # Process embeddings if we got enough
        if len(embeddings) >= MIN_EMBEDDINGS:
            # Average the embeddings
            avg_embedding = torch.mean(torch.stack(embeddings), dim=0)
            
            # Save to database
            db = load_embeddings()
            db[name] = avg_embedding
            save_embeddings(db)
            
            # Save a representative frame
            img_path = os.path.join("faces", f"{name}.jpg")
            cv2.imwrite(img_path, frame)
            
            messagebox.showinfo("Success", 
                              f"{name} registered successfully with {len(embeddings)} embeddings!")
        else:
            os.remove(video_path)
            messagebox.showerror("Error", 
                               f"Could only capture {len(embeddings)} embeddings. Please try again.")
    
    def recognize_faces(self):
        # Start the attendance session
        self.attendance_system.start_attendance_session()
        self.update_status("\nAttendance session started...")
        
        # Start updating the timer display
        self.update_timer()
        
        db = load_embeddings()
        if not db:
            messagebox.showinfo("Info", "No registered students found")
            return
        
        names = list(db.keys())
        embeddings = torch.stack(list(db.values())).squeeze(1)
        
        # Initialize camera
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Could not open video device")
            return
        
        self.update_status("Starting face recognition...")
        
        while True:
            # Check if we should stop (attendance period over)
            if not self.attendance_system.attendance_active:
                self.update_status("Attendance period completed. Session closed.")
                break
                
            ret, frame = self.cap.read()
            if not ret:
                self.update_status("Error reading frame from camera")
                break
            
            # Check if attendance interval has passed
            if self.attendance_system.check_attendance_time():
                session_data = self.attendance_system.finalize_attendance()
                self.update_status(f"Attendance marked at {datetime.now().strftime('%H:%M:%S')}")
                self.update_status(f"Present: {len(session_data['present'])}")
                self.update_status(f"Absent: {len(session_data['absent'])}")
                break
            
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(rgb_frame)
            boxes, _ = mtcnn.detect(img)
            
            if boxes is not None:
                faces = mtcnn(img)
                if faces is not None:
                    current_embeddings = resnet(faces.to(device)).detach().cpu()
                    
                    for i, (box, emb) in enumerate(zip(boxes, current_embeddings)):
                        x1, y1, x2, y2 = map(int, box)
                        
                        # Calculate distances to all known embeddings
                        dists = []
                        for known_emb in embeddings:
                            dist = torch.norm(known_emb - emb, dim=0)
                            dists.append(dist)
                        
                        # Use the minimum distance
                        min_dist = min(dists)
                        idx = dists.index(min_dist)
                        
                        # Lower threshold since we have better embeddings
                        if min_dist < 0.5:  # Reduced from 0.7
                            name = names[idx]
                            is_new = self.attendance_system.mark_present(name)
                            
                            if is_new:
                                self.update_status(f"{name} marked present at {datetime.now().strftime('%H:%M:%S')}")
                                label = f"{name} (Present)"
                                color = (0, 120, 255)  # Orange
                            else:
                                label = f"{name} (Already marked)"
                                color = (0, 180, 0)  # Dark green
                        else:
                            label = "Unknown"
                            color = (0, 0, 255)  # Red
                        
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(frame, label, (x1, y1-10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # Display time until next attendance marking
            if self.attendance_system.last_attendance_time:
                time_left = (self.attendance_system.last_attendance_time + 
                            timedelta(minutes=ATTENDANCE_INTERVAL) - datetime.now())
                mins, secs = divmod(max(0, time_left.seconds), 60)
                time_text = f"Time left: {mins:02d}:{secs:02d}"
                
                cv2.rectangle(frame, (5, 5), (250, 40), (0, 0, 0), -1)
                cv2.putText(frame, time_text, (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.imshow("Face Recognition - Press Q to Quit", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.attendance_system.attendance_active = False
                break
        
        self.cap.release()
        cv2.destroyAllWindows()
    
    def update_timer(self):
        if self.attendance_system.attendance_active and self.attendance_system.last_attendance_time:
            time_left = (self.attendance_system.last_attendance_time + 
                        timedelta(minutes=ATTENDANCE_INTERVAL) - datetime.now())
            if time_left.total_seconds() > 0:
                mins, secs = divmod(time_left.seconds, 60)
                self.time_label.config(text=f"Time left: {mins:02d}:{secs:02d}")
                self.master.after(1000, self.update_timer)
            else:
                self.time_label.config(text="Attendance session completed")
        else:
            self.time_label.config(text="Attendance not started")
    
    def manage_users(self):
        top = Toplevel(self.master)
        top.title("Student Management")
        top.geometry("900x600")
        top.configure(bg='white')
        
        main_frame = ttk.Frame(top)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Registered Students", 
                 style='Header.TLabel').pack(pady=10)
        
        # Create a canvas and scrollbar
        canvas = tk.Canvas(main_frame, bg='white')
        scrollbar = ttk.Scrollbar(main_frame, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Load registered users
        db = load_embeddings()
        users = list(db.keys())
        
        if not users:
            ttk.Label(scrollable_frame, text="No registered students").pack()
            return
        
        # Create a grid of student cards
        row_frame = None
        for i, user in enumerate(users):
            if i % 4 == 0:
                row_frame = ttk.Frame(scrollable_frame)
                row_frame.pack(fill='x', pady=5)
            
            card_frame = ttk.Frame(row_frame, style='Card.TFrame')
            card_frame.pack(side='left', padx=10, pady=5)
            
            # Student image
            img_path = os.path.join("faces", f"{user}.jpg")
            if os.path.exists(img_path):
                img = Image.open(img_path).resize((120, 120))
                photo = ImageTk.PhotoImage(img)
                lbl = ttk.Label(card_frame, image=photo)
                lbl.image = photo
                lbl.pack(pady=5)
            
            # Student name
            ttk.Label(card_frame, text=user, 
                     style='CardHeader.TLabel').pack()
            
            # Delete button
            ttk.Button(card_frame, text="Delete", 
                      command=lambda u=user: self.delete_user(u, top),
                      style='danger.TButton').pack(pady=5)
    
    def delete_user(self, user, window):
        if not messagebox.askyesno("Confirm", f"Delete {user} permanently?"):
            return
        
        db = load_embeddings()
        if user in db:
            del db[user]
            save_embeddings(db)
            
            img_path = os.path.join("faces", f"{user}.jpg")
            if os.path.exists(img_path):
                os.remove(img_path)
            
            video_path = os.path.join("faces", f"{user}.avi")
            if os.path.exists(video_path):
                os.remove(video_path)
            
            messagebox.showinfo("Success", f"Student {user} deleted successfully")
            window.destroy()
            self.manage_users()
        else:
            messagebox.showerror("Error", "Student not found")
    
    def view_attendance(self):
        top = Toplevel(self.master)
        top.title("Attendance Records")
        top.geometry("1000x600")
        
        main_frame = ttk.Frame(top)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Attendance History", 
                 style='Header.TLabel').pack(pady=10)
        
        # Create a text widget with scrollbar
        text_frame = ttk.Frame(main_frame)
        text_frame.pack(fill='both', expand=True)
        
        scrollbar = ttk.Scrollbar(text_frame)
        scrollbar.pack(side='right', fill='y')
        
        attendance_text = tk.Text(text_frame, wrap=tk.WORD, 
                                yscrollcommand=scrollbar.set,
                                font=('Courier New', 10))
        attendance_text.pack(fill='both', expand=True)
        
        scrollbar.config(command=attendance_text.yview)
        
        # Load and display attendance records
        if not os.path.exists(ATTENDANCE_FILE):
            attendance_text.insert(tk.END, "No attendance records found")
            return
        
        with open(ATTENDANCE_FILE, 'r') as f:
            records = json.load(f)
        
        for record in records:
            timestamp = record['timestamp']
            present = record['present']
            absent = record['absent']
            
            attendance_text.insert(tk.END, f"\n=== Session: {timestamp} ===\n")
            attendance_text.insert(tk.END, f"Present ({len(present)}):\n")
            for name, time in present.items():
                attendance_text.insert(tk.END, f"  - {name} at {time}\n")
            
            attendance_text.insert(tk.END, f"\nAbsent ({len(absent)}):\n")
            for name in absent:
                attendance_text.insert(tk.END, f"  - {name}\n")
            
            attendance_text.insert(tk.END, "\n" + "="*40 + "\n")
        
        attendance_text.config(state='disabled')
    
    def edit_attendance(self):
        """Method to manually edit attendance records"""
        edit_window = Toplevel(self.master)
        edit_window.title("Edit Attendance")
        edit_window.geometry("800x600")
        
        main_frame = ttk.Frame(edit_window)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Edit Attendance Records", 
                 style='Header.TLabel').pack(pady=10)
        
        # Date selection
        date_frame = ttk.Frame(main_frame)
        date_frame.pack(fill='x', pady=10)
        
        ttk.Label(date_frame, text="Select Session:").pack(side='left')
        
        # Load available sessions
        if not os.path.exists(ATTENDANCE_FILE):
            messagebox.showerror("Error", "No attendance records found")
            edit_window.destroy()
            return
        
        with open(ATTENDANCE_FILE, 'r') as f:
            records = json.load(f)
        
        session_dates = [r['timestamp'] for r in records]
        session_var = StringVar()
        session_dropdown = ttk.Combobox(date_frame, textvariable=session_var, 
                                       values=session_dates, state='readonly')
        session_dropdown.pack(side='left', padx=10)
        session_dropdown.current(0)
        
        # Student list frame with scrollbar
        student_frame = ttk.Frame(main_frame)
        student_frame.pack(fill='both', expand=True)
        
        # Container for student status variables
        student_status_vars = {}
        
        def load_session():
            selected_session = session_var.get()
            if not selected_session:
                return
            
            # Find the selected session
            session_data = next((r for r in records if r['timestamp'] == selected_session), None)
            if not session_data:
                return
            
            # Clear previous entries
            for widget in student_frame.winfo_children():
                widget.destroy()
            
            # Get all registered students
            db = load_embeddings()
            all_students = list(db.keys())
            
            # Create attendance list
            for student in all_students:
                row_frame = ttk.Frame(student_frame)
                row_frame.pack(fill='x', pady=2)
                
                # Student name
                ttk.Label(row_frame, text=student, width=20).pack(side='left')
                
                # Attendance status
                status_var = StringVar()
                status_var.set('present' if student in session_data['present'] else 'absent')
                
                ttk.Radiobutton(row_frame, text="Present", variable=status_var, 
                               value='present').pack(side='left', padx=5)
                ttk.Radiobutton(row_frame, text="Absent", variable=status_var, 
                               value='absent').pack(side='left', padx=5)
                
                # Store reference to update later
                student_status_vars[student] = status_var
        
        ttk.Button(date_frame, text="Load Session", command=load_session, 
                  style='primary.TButton').pack(side='left')
        
        def save_changes():
            selected_session = session_var.get()
            if not selected_session:
                return
            
            # Find the session to update
            session_index = next((i for i, r in enumerate(records) 
                                if r['timestamp'] == selected_session), None)
            if session_index is None:
                return
            
            # Update present/absent lists based on current selections
            present = {}
            absent = []
            
            for student, var in student_status_vars.items():
                if var.get() == 'present':
                    # Try to preserve original timestamp if available
                    original_time = records[session_index]['present'].get(student, 
                        datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    present[student] = original_time
                else:
                    absent.append(student)
            
            # Update the record
            records[session_index]['present'] = present
            records[session_index]['absent'] = absent
            
            # Save back to file
            with open(ATTENDANCE_FILE, 'w') as f:
                json.dump(records, f, indent=4)
            
            messagebox.showinfo("Success", "Attendance records updated successfully")
        
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=10)
        
        ttk.Button(button_frame, text="Save Changes", command=save_changes,
                  style='success.TButton').pack(side='left', padx=5)
        ttk.Button(button_frame, text="Cancel", command=edit_window.destroy,
                  style='danger.TButton').pack(side='left', padx=5)
        
        # Load the first session by default
        load_session()

if __name__ == "__main__":
    root = Tk()
    app = FaceAttendanceApp(root)
    root.mainloop()

c:\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception in thread Thread-3 (update_progress):
Traceback (most recent call last):
  File "c:\Python\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\anany\AppData\Roaming\Python\Python310\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Python\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\anany\AppData\Local\Temp\ipykernel_4048\369273122.py", line 291, in update_progress
  File "c:\Python\lib\tkinter\__init__.py", line 1675, in configure
    return self._configure('configure', cnf, kw)
  File "c:\Python\lib\tkinter\__init__.py", line 1665, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))

In [5]:
pip install onnxruntime-gpu


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/213.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/213.1 MB ? eta -:--:--
     ---------------------------------------- 0.1/213.1 MB 1.1 MB/s eta 0:03:16
     ---------------------------------------- 0.1/213.1 MB 1.2 MB/s eta 0:02:55
     ---------------------------------------- 0.2/213.1 MB 1.4 MB/s eta 0:02:35
     ---------------------------------------- 0.3/213.1 MB 1.5 MB/s eta 0:02:26
     ---------------------------------------- 0.4/213.1 MB 1.6 MB/s eta 0:02:14
     ---------------------------------------- 0.5/213.1 MB 1.7 MB/s eta 0:02:09
     ---------------------------------------- 0.5/213.1 MB 1.4 MB/s eta 0:02:29
     ---------------------------------------- 0.6/213.1 MB 1.6 MB/s eta 0:02:15
     ---------------------------------------- 0.7/213.1 MB 1.6 MB/s eta 0:02:17
     ---------------------------------------- 0.8/213.1 MB 1.6 MB/s eta 0:02:10
     ---------------------------------------- 0.8/213

In [7]:
pip install numpy onnxruntime-gpu opencv-python Pillow requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'D:\\Ananya\\Projects'
Hint: It looks like a path. File 'D:\Ananya\Projects' does not exist.

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install insightface --upgrade

  Using cached insightface-0.7.3.tar.gz (439 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached Cython-3.0.12-cp310-cp310-win_amd64.whl (2.8 MB)
  Using cached easydict-1.13-py3-none-any.whl (6.8 kB)
  Using cached onnx-1.17.0-cp310-cp310-win_amd64.whl (14.5 MB)
  Using cached prettytable-3.16.0-py3-none-any.whl (33 kB)
  Using cached scikit_image-0.25.2-cp310-cp310-win_amd64.whl (12.8 MB)
  Using cached albumentations-2.0.5-py3-none-any.whl (290 kB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)
  Using cached albucore-0.0.23-py3-none-any.whl (14 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
  Using cached simsimd-6.2.1-cp310-cp310-win_amd64.w

  error: subprocess-exited-with-error
  
  × Building wheel for insightface (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [213 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-310\insightface
      copying insightface\__init__.py -> build\lib.win-amd64-cpython-310\insightface
      creating build\lib.win-amd64-cpython-310\insightface\app
      copying insightface\app\common.py -> build\lib.win-amd64-cpython-310\insightface\app
      copying insightface\app\face_analysis.py -> build\lib.win-amd64-cpython-310\insightface\app
      copying insightface\app\mask_renderer.py -> build\lib.win-amd64-cpython-310\insightface\app
      copying insightface\app\__init__.py -> build\lib.win-amd64-cpython-310\insightface\app
      creating build\lib.win-amd64-cpython-310\insightface\commands
      copying insightface\commands\insightface_cli.py -> build\lib.win-amd64-cpython-310\insightface\commands
  

In [2]:
import cv2
import os
import torch
import pickle
import json
import numpy as np
from datetime import datetime, timedelta
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk, Tk, Label, Button, Entry, StringVar, Toplevel, Frame, messagebox, scrolledtext
from tkinter import filedialog
from facenet_pytorch import MTCNN, InceptionResnetV1
import time
from threading import Thread
from insightface.app import FaceAnalysis  # New improved model

# Configuration
os.makedirs("faces", exist_ok=True)
os.makedirs("attendance", exist_ok=True)
EMBEDDING_PATH = "face_embeddings.pkl"
ATTENDANCE_FILE = "attendance/attendance_records.json"
ATTENDANCE_INTERVAL = 1  # minutes
VIDEO_LENGTH = 15  # seconds
MIN_EMBEDDINGS = 30  # Minimum embeddings to capture
LIVENESS_THRESHOLD = 0.7  # Threshold for liveness detection

class FaceRecognizer:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.detector = MTCNN(image_size=160, margin=20, min_face_size=40, 
                             device=self.device, keep_all=True)
        self.recognizer = InceptionResnetV1(pretrained='vggface2').eval().to(self.device)
        
    def get_embeddings(self, image):
        faces = self.detector(image)
        if faces is not None:
            return self.recognizer(faces.to(self.device)).detach().cpu()
        return None

class ImprovedFaceRecognizer:
    def __init__(self):
        self.app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
        self.app.prepare(ctx_id=0, det_size=(640, 640))
        
    def detect_faces(self, frame):
        # Preprocess image first
        frame = self.preprocess_image(frame)
        faces = self.app.get(frame)
        return faces
        
    def preprocess_image(self, frame):
        # Contrast enhancement
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl,a,b))
        frame = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        # Denoising
        frame = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
        return frame

class LivenessDetector:
    def __init__(self):
        # Load liveness detection model
        self.model = self.load_liveness_model()
        
    def load_liveness_model(self):
        # Placeholder - implement actual liveness detection
        return None
        
    def is_live(self, frame, face_region):
        # Implement actual liveness check (motion, texture analysis, etc.)
        # For now return True by default
        return True

class FaceTracker:
    def __init__(self):
        self.tracked_faces = {}
        self.next_id = 0
        
    def update(self, detections):
        # Simple tracking by proximity - replace with DeepSort or similar for better tracking
        updated_tracks = []
        
        for det in detections:
            matched = False
            for face_id, face_data in self.tracked_faces.items():
                if self._is_same_face(face_data['bbox'], det['bbox']):
                    # Update existing track
                    self.tracked_faces[face_id] = det
                    det['id'] = face_id
                    updated_tracks.append(det)
                    matched = True
                    break
                    
            if not matched:
                # New face
                new_id = self.next_id
                self.next_id += 1
                det['id'] = new_id
                self.tracked_faces[new_id] = det
                updated_tracks.append(det)
                
        # Remove stale tracks
        self._cleanup_tracks()
        return updated_tracks
        
    def _is_same_face(self, box1, box2):
        # Simple IoU calculation for tracking
        x1, y1, x2, y2 = box1
        x1_, y1_, x2_, y2_ = box2
        
        xi1 = max(x1, x1_)
        yi1 = max(y1, y1_)
        xi2 = min(x2, x2_)
        yi2 = min(y2, y2_)
        
        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
        box1_area = (x2 - x1) * (y2 - y1)
        box2_area = (x2_ - x1_) * (y2_ - y1_)
        
        iou = inter_area / float(box1_area + box2_area - inter_area)
        return iou > 0.3  # Threshold for considering same face
        
    def _cleanup_tracks(self):
        # Remove faces that haven't been seen in a while
        pass

class AttendanceSystem:
    def __init__(self):
        self.attendance_records = []
        self.current_session = {}
        self.last_attendance_time = None
        self.attendance_active = False
        self.recognizer = ImprovedFaceRecognizer()
        self.liveness_detector = LivenessDetector()
        self.tracker = FaceTracker()
        self.load_attendance()
    
    def load_attendance(self):
        if os.path.exists(ATTENDANCE_FILE):
            with open(ATTENDANCE_FILE, 'r') as f:
                self.attendance_records = json.load(f)
    
    def save_attendance(self):
        with open(ATTENDANCE_FILE, 'w') as f:
            json.dump(self.attendance_records, f, indent=4)
    
    def mark_present(self, name, frame=None, face_region=None):
        # Check liveness if frame and face_region are provided
        if frame is not None and face_region is not None:
            if not self.liveness_detector.is_live(frame, face_region):
                self.log_spoof_attempt(name)
                return False
        
        now = datetime.now()
        time_str = now.strftime("%Y-%m-%d %H:%M:%S")
        
        if name not in self.current_session:
            self.current_session[name] = time_str
            return True
        return False
    
    def log_spoof_attempt(self, name):
        # Log spoofing attempts for security review
        log_file = "attendance/spoof_attempts.log"
        with open(log_file, 'a') as f:
            f.write(f"Spoof attempt detected for {name} at {datetime.now()}\n")
    
    def check_attendance_time(self):
        if not self.attendance_active or self.last_attendance_time is None:
            return False
            
        now = datetime.now()
        if (now - self.last_attendance_time) >= timedelta(minutes=ATTENDANCE_INTERVAL):
            self.finalize_attendance()
            self.last_attendance_time = None
            self.attendance_active = False
            return True
        return False
    
    def start_attendance_session(self):
        self.last_attendance_time = datetime.now()
        self.attendance_active = True
        self.current_session = {}
    
    def finalize_attendance(self):
        if not self.current_session:
            return
            
        db = self.load_embeddings()
        all_names = set(db.keys())
        present_names = set(self.current_session.keys())
        absent_names = all_names - present_names
        
        session_data = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "present": {name: self.current_session[name] for name in present_names},
            "absent": list(absent_names)
        }
        
        self.attendance_records.append(session_data)
        self.save_attendance()
        self.current_session = {}
        return session_data
    
    def identify_face(self, embedding):
        db = self.load_embeddings()
        if not db:
            return None
            
        min_dist = float('inf')
        identity = None
        
        for name, known_emb in db.items():
            dist = torch.norm(known_emb - embedding, dim=0)
            if dist < min_dist and dist < 0.5:  # Threshold
                min_dist = dist
                identity = name
                
        return identity
    
    def load_embeddings(self):
        if os.path.exists(EMBEDDING_PATH):
            with open(EMBEDDING_PATH, 'rb') as f:
                return pickle.load(f)
        return {}
    
    def save_embeddings(self, data):
        with open(EMBEDDING_PATH, 'wb') as f:
            pickle.dump(data, f)

class FaceAttendanceApp:
    def __init__(self, master):
        self.master = master
        self.attendance_system = AttendanceSystem()
        self.cap = None  # VideoCapture object
        
        # Window configuration
        master.title("Smart Attendance System")
        master.geometry("1000x700")
        master.configure(bg='#f5f5f5')
        
        # Custom style
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Color scheme
        self.colors = {
            'primary': '#4e73df',
            'success': '#1cc88a',
            'info': '#36b9cc',
            'warning': '#f6c23e',
            'danger': '#e74a3b',
            'light': '#f8f9fc',
            'dark': '#5a5c69'
        }
        
        # Configure styles
        self.configure_styles()
        
        # Header frame
        header_frame = ttk.Frame(master, style='Header.TFrame')
        header_frame.pack(fill='x', padx=10, pady=10)
        
        ttk.Label(header_frame, text="Smart Attendance System", 
                 style='Header.TLabel').pack(pady=15)
        
        # Main content frame
        main_frame = ttk.Frame(master)
        main_frame.pack(fill='both', expand=True, padx=20, pady=10)
        
        # Left panel - Controls
        control_frame = ttk.Frame(main_frame, style='Card.TFrame')
        control_frame.pack(side='left', fill='y', padx=(0, 10))
        
        ttk.Label(control_frame, text="System Controls", 
                 style='CardHeader.TLabel').pack(pady=10)        
        control_buttons = [
            ("Add New Student", self.add_face_gui, 'primary'),
            ("Start Attendance", self.recognize_faces, 'success'),
            ("Manage Students", self.manage_users, 'info'),
            ("View Attendance", self.view_attendance, 'warning'),
            ("Edit Attendance", self.edit_attendance, 'primary'),
            ("Attendance Analytics", self.show_analytics, 'info')  # New feature
        ]
        
        for text, command, color in control_buttons:
            btn = ttk.Button(control_frame, text=text, command=command,
                            style=f'{color}.TButton')
            btn.pack(fill='x', padx=10, pady=5)
        
        # Right panel - Status
        status_frame = ttk.Frame(main_frame, style='Card.TFrame')
        status_frame.pack(side='right', fill='both', expand=True)
        
        ttk.Label(status_frame, text="Attendance Status", 
                 style='CardHeader.TLabel').pack(pady=10)
        
        self.status_text = scrolledtext.ScrolledText(status_frame, 
                                                   height=20,
                                                   font=('Helvetica', 10),
                                                   wrap=tk.WORD)
        self.status_text.pack(fill='both', expand=True, padx=10, pady=5)
        
        # Footer
        footer_frame = ttk.Frame(master, style='Footer.TFrame')
        footer_frame.pack(fill='x', padx=10, pady=10)
        
        self.time_label = ttk.Label(footer_frame, 
                 text="Attendance not started",
                 style='Footer.TLabel')
        self.time_label.pack()
    
    def configure_styles(self):
        # ... (same as before) ...
        self.style.configure('Header.TFrame', background=self.colors['primary'])
        self.style.configure('Header.TLabel', 
                           background=self.colors['primary'],
                           foreground='white',
                           font=('Helvetica', 18, 'bold'))
        
        self.style.configure('Card.TFrame', 
                           background='white',
                           relief='groove',
                           borderwidth=2)
        
        self.style.configure('CardHeader.TLabel',
                           font=('Helvetica', 12, 'bold'),
                           background='white',
                           foreground=self.colors['dark'])
        
        self.style.configure('Footer.TFrame',
                           background=self.colors['light'])
        
        self.style.configure('Footer.TLabel',
                           background=self.colors['light'],
                           font=('Helvetica', 10))
        
        for color_name, color_code in self.colors.items():
            if color_name in ['primary', 'success', 'info', 'warning', 'danger']:
                self.style.configure(f'{color_name}.TButton',
                                    foreground='white',
                                    background=color_code,
                                    font=('Helvetica', 10, 'bold'),
                                    padding=8)
                
                self.style.map(f'{color_name}.TButton',
                              background=[('active', color_code), ('disabled', '#dddddd')])
    
    def update_status(self, message):
        self.status_text.insert(tk.END, message + "\n")
        self.status_text.see(tk.END)
        self.status_text.update()
    
    def add_face_gui(self):
        # ... (same as before) ...
        def start_video_capture():
            name = name_var.get().strip()
            if not name:
                messagebox.showerror("Error", "Please enter a valid name")
                return
            
            top.destroy()
            self.capture_video_embeddings(name)
        
        top = Toplevel(self.master)
        top.title("Register New Student")
        top.geometry("350x200")
        top.configure(bg='white')
        
        ttk.Label(top, text="Enter Student Name:", 
                 style='CardHeader.TLabel').pack(pady=10)
        
        name_var = StringVar()
        name_entry = ttk.Entry(top, textvariable=name_var, font=('Helvetica', 12))
        name_entry.pack(pady=5, padx=20, fill='x')
        
        btn_frame = ttk.Frame(top)
        btn_frame.pack(pady=10)
        
        ttk.Button(btn_frame, text="Record Video", 
                  command=start_video_capture, style='primary.TButton').pack(side='left', padx=5)
        ttk.Button(btn_frame, text="Cancel", 
                  command=top.destroy, style='danger.TButton').pack(side='left', padx=5)
    
    def capture_video_embeddings(self, name):
        """Capture video and extract multiple face embeddings using improved model"""
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Could not open video device")
            return
            
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        video_path = os.path.join("faces", f"{name}.avi")
        out = cv2.VideoWriter(video_path, fourcc, 20.0, (640, 480))
        
        embeddings = []
        start_time = time.time()
        capture_complete = False
        
        # Create a progress window
        progress_window = Toplevel(self.master)
        progress_window.title("Recording Video")
        progress_window.geometry("400x150")
        
        progress_label = ttk.Label(progress_window, text="Please move your head slowly in different directions")
        progress_label.pack(pady=10)
        
        progress_var = tk.DoubleVar()
        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, 
                                      maximum=VIDEO_LENGTH, length=350)
        progress_bar.pack(pady=10)
        
        time_label = ttk.Label(progress_window, text=f"Time remaining: {VIDEO_LENGTH} seconds")
        time_label.pack()
        
        def update_progress():
            while not capture_complete:
                elapsed = time.time() - start_time
                remaining = max(0, VIDEO_LENGTH - elapsed)
                progress_var.set(elapsed)
                time_label.config(text=f"Time remaining: {int(remaining)} seconds")
                if elapsed >= VIDEO_LENGTH:
                    break
                time.sleep(0.1)
        
        # Start progress update thread
        progress_thread = Thread(target=update_progress)
        progress_thread.start()
        
        # Main capture loop
        while (time.time() - start_time) < VIDEO_LENGTH:
            ret, frame = self.cap.read()
            if not ret:
                continue
            
            # Write frame to video file
            out.write(frame)
            
            # Process frame with improved face detection
            faces = self.attendance_system.recognizer.detect_faces(frame)
            
            if faces:
                for face in faces:
                    embeddings.append(face['embedding'])
            
            # Display instructions and count
            cv2.putText(frame, "Move your head slowly in different directions", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Embeddings captured: {len(embeddings)}/{MIN_EMBEDDINGS}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Time remaining: {int(VIDEO_LENGTH - (time.time() - start_time))}s", 
                        (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.imshow("Recording Video - Move Your Head", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            # Early exit if we have enough embeddings
            if len(embeddings) >= MIN_EMBEDDINGS:
                break
        
        # Clean up
        capture_complete = True
        self.cap.release()
        out.release()
        cv2.destroyAllWindows()
        progress_window.destroy()
        
        # Process embeddings if we got enough
        if len(embeddings) >= MIN_EMBEDDINGS:
            # Average the embeddings
            avg_embedding = torch.mean(torch.stack(embeddings), dim=0)
            
            # Save to database
            db = self.attendance_system.load_embeddings()
            db[name] = avg_embedding
            self.attendance_system.save_embeddings(db)
            
            # Save a representative frame
            img_path = os.path.join("faces", f"{name}.jpg")
            cv2.imwrite(img_path, frame)
            
            messagebox.showinfo("Success", 
                              f"{name} registered successfully with {len(embeddings)} embeddings!")
        else:
            os.remove(video_path)
            messagebox.showerror("Error", 
                               f"Could only capture {len(embeddings)} embeddings. Please try again.")
    
    def recognize_faces(self):
        """Improved face recognition with tracking and liveness detection"""
        self.attendance_system.start_attendance_session()
        self.update_status("\nAttendance session started...")
        self.update_timer()
        
        db = self.attendance_system.load_embeddings()
        if not db:
            messagebox.showinfo("Info", "No registered students found")
            return
        
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Could not open video device")
            return
        
        self.update_status("Starting face recognition...")
        
        while True:
            if not self.attendance_system.attendance_active:
                self.update_status("Attendance period completed. Session closed.")
                break
                
            ret, frame = self.cap.read()
            if not ret:
                self.update_status("Error reading frame from camera")
                break
            
            # Check if attendance interval has passed
            if self.attendance_system.check_attendance_time():
                session_data = self.attendance_system.finalize_attendance()
                self.update_status(f"Attendance marked at {datetime.now().strftime('%H:%M:%S')}")
                self.update_status(f"Present: {len(session_data['present'])}")
                self.update_status(f"Absent: {len(session_data['absent'])}")
                break
            
            # Detect faces with improved recognizer
            faces = self.attendance_system.recognizer.detect_faces(frame)
            tracked_faces = self.attendance_system.tracker.update(faces)
            
            for face in tracked_faces:
                x1, y1, x2, y2 = map(int, face['bbox'])
                face_region = frame[y1:y2, x1:x2]
                
                # Identify face
                identity = self.attendance_system.identify_face(face['embedding'])
                
                if identity:
                    # Check liveness and mark present
                    is_new = self.attendance_system.mark_present(identity, frame, face_region)
                    
                    if is_new:
                        self.update_status(f"{identity} marked present at {datetime.now().strftime('%H:%M:%S')}")
                        label = f"{identity} (Present)"
                        color = (0, 120, 255)  # Orange
                    else:
                        label = f"{identity} (Already marked)"
                        color = (0, 180, 0)  # Dark green
                else:
                    label = "Unknown"
                    color = (0, 0, 255)  # Red
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1-10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # Display time until next attendance marking
            if self.attendance_system.last_attendance_time:
                time_left = (self.attendance_system.last_attendance_time + 
                            timedelta(minutes=ATTENDANCE_INTERVAL) - datetime.now())
                mins, secs = divmod(max(0, time_left.seconds), 60)
                time_text = f"Time left: {mins:02d}:{secs:02d}"
                
                cv2.rectangle(frame, (5, 5), (250, 40), (0, 0, 0), -1)
                cv2.putText(frame, time_text, (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.imshow("Face Recognition - Press Q to Quit", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.attendance_system.attendance_active = False
                break
        
        self.cap.release()
        cv2.destroyAllWindows()
    
    def update_timer(self):
        # ... (same as before) ...
        if self.attendance_system.attendance_active and self.attendance_system.last_attendance_time:
            time_left = (self.attendance_system.last_attendance_time + 
                        timedelta(minutes=ATTENDANCE_INTERVAL) - datetime.now())
            if time_left.total_seconds() > 0:
                mins, secs = divmod(time_left.seconds, 60)
                self.time_label.config(text=f"Time left: {mins:02d}:{secs:02d}")
                self.master.after(1000, self.update_timer)
            else:
                self.time_label.config(text="Attendance session completed")
        else:
            self.time_label.config(text="Attendance not started")
    
    def manage_users(self):
        # ... (same as before) ...
        top = Toplevel(self.master)
        top.title("Student Management")
        top.geometry("900x600")
        top.configure(bg='white')
        
        main_frame = ttk.Frame(top)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Registered Students", 
                 style='Header.TLabel').pack(pady=10)
        
        # Create a canvas and scrollbar
        canvas = tk.Canvas(main_frame, bg='white')
        scrollbar = ttk.Scrollbar(main_frame, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Load registered users
        db = load_embeddings()
        users = list(db.keys())
        
        if not users:
            ttk.Label(scrollable_frame, text="No registered students").pack()
            return
        
        # Create a grid of student cards
        row_frame = None
        for i, user in enumerate(users):
            if i % 4 == 0:
                row_frame = ttk.Frame(scrollable_frame)
                row_frame.pack(fill='x', pady=5)
            
            card_frame = ttk.Frame(row_frame, style='Card.TFrame')
            card_frame.pack(side='left', padx=10, pady=5)
            
            # Student image
            img_path = os.path.join("faces", f"{user}.jpg")
            if os.path.exists(img_path):
                img = Image.open(img_path).resize((120, 120))
                photo = ImageTk.PhotoImage(img)
                lbl = ttk.Label(card_frame, image=photo)
                lbl.image = photo
                lbl.pack(pady=5)
            
            # Student name
            ttk.Label(card_frame, text=user, 
                     style='CardHeader.TLabel').pack()
            
            # Delete button
            ttk.Button(card_frame, text="Delete", 
                      command=lambda u=user: self.delete_user(u, top),
                      style='danger.TButton').pack(pady=5)
    
    def delete_user(self, user, window):
        # ... (same as before) ...
        if not messagebox.askyesno("Confirm", f"Delete {user} permanently?"):
            return
        
        db = load_embeddings()
        if user in db:
            del db[user]
            save_embeddings(db)
            
            img_path = os.path.join("faces", f"{user}.jpg")
            if os.path.exists(img_path):
                os.remove(img_path)
            
            video_path = os.path.join("faces", f"{user}.avi")
            if os.path.exists(video_path):
                os.remove(video_path)
            
            messagebox.showinfo("Success", f"Student {user} deleted successfully")
            window.destroy()
            self.manage_users()
        else:
            messagebox.showerror("Error", "Student not found")
    
    def view_attendance(self):
        # ... (same as before) ...
        top = Toplevel(self.master)
        top.title("Attendance Records")
        top.geometry("1000x600")
        
        main_frame = ttk.Frame(top)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Attendance History", 
                 style='Header.TLabel').pack(pady=10)
        
        # Create a text widget with scrollbar
        text_frame = ttk.Frame(main_frame)
        text_frame.pack(fill='both', expand=True)
        
        scrollbar = ttk.Scrollbar(text_frame)
        scrollbar.pack(side='right', fill='y')
        
        attendance_text = tk.Text(text_frame, wrap=tk.WORD, 
                                yscrollcommand=scrollbar.set,
                                font=('Courier New', 10))
        attendance_text.pack(fill='both', expand=True)
        
        scrollbar.config(command=attendance_text.yview)
        
        # Load and display attendance records
        if not os.path.exists(ATTENDANCE_FILE):
            attendance_text.insert(tk.END, "No attendance records found")
            return
        
        with open(ATTENDANCE_FILE, 'r') as f:
            records = json.load(f)
        
        for record in records:
            timestamp = record['timestamp']
            present = record['present']
            absent = record['absent']
            
            attendance_text.insert(tk.END, f"\n=== Session: {timestamp} ===\n")
            attendance_text.insert(tk.END, f"Present ({len(present)}):\n")
            for name, time in present.items():
                attendance_text.insert(tk.END, f"  - {name} at {time}\n")
            
            attendance_text.insert(tk.END, f"\nAbsent ({len(absent)}):\n")
            for name in absent:
                attendance_text.insert(tk.END, f"  - {name}\n")
            
            attendance_text.insert(tk.END, "\n" + "="*40 + "\n")
        
        attendance_text.config(state='disabled')
    
    def edit_attendance(self):
        # ... (same as before) ...
        edit_window = Toplevel(self.master)
        edit_window.title("Edit Attendance")
        edit_window.geometry("800x600")
        
        main_frame = ttk.Frame(edit_window)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Edit Attendance Records", 
                 style='Header.TLabel').pack(pady=10)
        
        # Date selection
        date_frame = ttk.Frame(main_frame)
        date_frame.pack(fill='x', pady=10)
        
        ttk.Label(date_frame, text="Select Session:").pack(side='left')
        
        # Load available sessions
        if not os.path.exists(ATTENDANCE_FILE):
            messagebox.showerror("Error", "No attendance records found")
            edit_window.destroy()
            return
        
        with open(ATTENDANCE_FILE, 'r') as f:
            records = json.load(f)
        
        session_dates = [r['timestamp'] for r in records]
        session_var = StringVar()
        session_dropdown = ttk.Combobox(date_frame, textvariable=session_var, 
                                       values=session_dates, state='readonly')
        session_dropdown.pack(side='left', padx=10)
        session_dropdown.current(0)
        
        # Student list frame with scrollbar
        student_frame = ttk.Frame(main_frame)
        student_frame.pack(fill='both', expand=True)
        
        # Container for student status variables
        student_status_vars = {}
        
        def load_session():
            selected_session = session_var.get()
            if not selected_session:
                return
            
            # Find the selected session
            session_data = next((r for r in records if r['timestamp'] == selected_session), None)
            if not session_data:
                return
            
            # Clear previous entries
            for widget in student_frame.winfo_children():
                widget.destroy()
            
            # Get all registered students
            db = load_embeddings()
            all_students = list(db.keys())
            
            # Create attendance list
            for student in all_students:
                row_frame = ttk.Frame(student_frame)
                row_frame.pack(fill='x', pady=2)
                
                # Student name
                ttk.Label(row_frame, text=student, width=20).pack(side='left')
                
                # Attendance status
                status_var = StringVar()
                status_var.set('present' if student in session_data['present'] else 'absent')
                
                ttk.Radiobutton(row_frame, text="Present", variable=status_var, 
                               value='present').pack(side='left', padx=5)
                ttk.Radiobutton(row_frame, text="Absent", variable=status_var, 
                               value='absent').pack(side='left', padx=5)
                
                # Store reference to update later
                student_status_vars[student] = status_var
        
        ttk.Button(date_frame, text="Load Session", command=load_session, 
                  style='primary.TButton').pack(side='left')
        
        def save_changes():
            selected_session = session_var.get()
            if not selected_session:
                return
            
            # Find the session to update
            session_index = next((i for i, r in enumerate(records) 
                                if r['timestamp'] == selected_session), None)
            if session_index is None:
                return
            
            # Update present/absent lists based on current selections
            present = {}
            absent = []
            
            for student, var in student_status_vars.items():
                if var.get() == 'present':
                    # Try to preserve original timestamp if available
                    original_time = records[session_index]['present'].get(student, 
                        datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    present[student] = original_time
                else:
                    absent.append(student)
            
            # Update the record
            records[session_index]['present'] = present
            records[session_index]['absent'] = absent
            
            # Save back to file
            with open(ATTENDANCE_FILE, 'w') as f:
                json.dump(records, f, indent=4)
            
            messagebox.showinfo("Success", "Attendance records updated successfully")
        
        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=10)
        
        ttk.Button(button_frame, text="Save Changes", command=save_changes,
                  style='success.TButton').pack(side='left', padx=5)
        ttk.Button(button_frame, text="Cancel", command=edit_window.destroy,
                  style='danger.TButton').pack(side='left', padx=5)
        
        # Load the first session by default
        load_session()
    
    def show_analytics(self):
        """New method to display attendance analytics"""
        analytics_window = Toplevel(self.master)
        analytics_window.title("Attendance Analytics")
        analytics_window.geometry("1000x700")
        
        # Load attendance data
        if not os.path.exists(ATTENDANCE_FILE):
            messagebox.showerror("Error", "No attendance records found")
            analytics_window.destroy()
            return
        
        with open(ATTENDANCE_FILE, 'r') as f:
            records = json.load(f)
        
        # Create analytics tabs
        notebook = ttk.Notebook(analytics_window)
        
        # Tab 1: Summary Statistics
        summary_frame = ttk.Frame(notebook)
        self.create_summary_tab(summary_frame, records)
        notebook.add(summary_frame, text="Summary")
        
        # Tab 2: Attendance Trends
        trends_frame = ttk.Frame(notebook)
        self.create_trends_tab(trends_frame, records)
        notebook.add(trends_frame, text="Trends")
        
        # Tab 3: Individual Reports
        individual_frame = ttk.Frame(notebook)
        self.create_individual_tab(individual_frame, records)
        notebook.add(individual_frame, text="Individual Reports")
        
        notebook.pack(expand=True, fill='both')
    
    def create_summary_tab(self, frame, records):
        # Implement summary statistics visualization
        pass
    
    def create_trends_tab(self, frame, records):
        # Implement attendance trends visualization
        pass
    
    def create_individual_tab(self, frame, records):
        # Implement individual attendance reports
        pass

if __name__ == "__main__":
    root = Tk()
    app = FaceAttendanceApp(root)
    root.mainloop()

c:\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'insightface'

Better Version(Previous One) without InsightFace

In [3]:
import warnings
from tqdm import TqdmWarning

# Suppress tqdm IProgress warning
warnings.filterwarnings("ignore", category=TqdmWarning)

import cv2
import os
import torch
import pickle
import json
import numpy as np
from datetime import datetime, timedelta
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk, Tk, Label, Button, Entry, StringVar, Toplevel, Frame, messagebox, scrolledtext
from tkinter import filedialog
from facenet_pytorch import MTCNN, InceptionResnetV1
import time
from threading import Thread

# Configuration
os.makedirs("faces", exist_ok=True)
os.makedirs("attendance", exist_ok=True)
EMBEDDING_PATH = "face_embeddings.pkl"
ATTENDANCE_FILE = "attendance/attendance_records.json"
ATTENDANCE_INTERVAL = 1  # minutes
VIDEO_LENGTH = 15  # seconds
MIN_EMBEDDINGS = 30  # Minimum embeddings to capture

class FaceRecognizer:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Using device: {self.device}")
        
        # Initialize MTCNN for face detection
        self.detector = MTCNN(
            image_size=160, 
            margin=20, 
            min_face_size=40,
            thresholds=[0.6, 0.7, 0.7],  # MTCNN thresholds
            factor=0.709, 
            post_process=True,
            device=self.device,
            keep_all=True
        )
        
        # Initialize InceptionResnetV1 for face recognition
        self.recognizer = InceptionResnetV1(
            pretrained='vggface2',
            classify=False,
            device=self.device
        ).eval()
        
    def preprocess_image(self, frame):
        """Enhance image quality for better face detection"""
        # Convert to LAB color space for contrast enhancement
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l_channel, a, b = cv2.split(lab)
        
        # Apply CLAHE to L channel
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l_channel)
        
        # Merge the CLAHE enhanced L channel back
        limg = cv2.merge((cl, a, b))
        
        # Convert back to BGR
        enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        # Apply mild denoising
        denoised = cv2.fastNlMeansDenoisingColored(enhanced, None, 10, 10, 7, 21)
        
        return denoised
    
    def detect_faces(self, frame):
        """Detect faces in a frame and return bounding boxes and embeddings"""
        # Preprocess the image
        processed_frame = self.preprocess_image(frame)
        
        # Convert to PIL Image (required by MTCNN)
        img = Image.fromarray(cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB))
        
        # Detect faces
        boxes, probs = self.detector.detect(img)
        
        faces = []
        if boxes is not None:
            # Get face embeddings for each detected face
            face_tensors = self.detector(img)
            if face_tensors is not None:
                embeddings = self.recognizer(face_tensors.to(self.device)).detach().cpu()
                
                for i, box in enumerate(boxes):
                    faces.append({
                        'bbox': box,
                        'embedding': embeddings[i] if i < len(embeddings) else None,
                        'confidence': probs[i] if probs is not None else 0.9
                    })
        
        return faces

class FaceTracker:
    def __init__(self):
        self.tracked_faces = {}
        self.next_id = 0
        
    def update(self, detections):
        updated_tracks = []
        
        for det in detections:
            matched = False
            x1, y1, x2, y2 = det['bbox']
            center = ((x1 + x2) / 2, (y1 + y2) / 2)
            
            for face_id, face_data in self.tracked_faces.items():
                # Simple distance-based tracking
                prev_center = face_data['center']
                distance = np.sqrt((center[0] - prev_center[0])**2 + (center[1] - prev_center[1])**2)
                
                # If centers are close, consider it the same face
                if distance < 50:  # pixels threshold
                    # Update the track
                    self.tracked_faces[face_id] = {
                        'bbox': det['bbox'],
                        'center': center,
                        'embedding': det.get('embedding'),
                        'confidence': det.get('confidence', 0.9)
                    }
                    det['id'] = face_id
                    updated_tracks.append(det)
                    matched = True
                    break
                    
            if not matched:
                # New face
                new_id = self.next_id
                self.next_id += 1
                det['id'] = new_id
                self.tracked_faces[new_id] = {
                    'bbox': det['bbox'],
                    'center': center,
                    'embedding': det.get('embedding'),
                    'confidence': det.get('confidence', 0.9)
                }
                updated_tracks.append(det)
                
        # Remove stale tracks (not seen in this frame)
        active_ids = {det['id'] for det in updated_tracks}
        self.tracked_faces = {k: v for k, v in self.tracked_faces.items() if k in active_ids}
        
        return updated_tracks

class AttendanceSystem:
    def __init__(self):
        self.attendance_records = []
        self.current_session = {}
        self.last_attendance_time = None
        self.attendance_active = False
        self.recognizer = FaceRecognizer()
        self.tracker = FaceTracker()
        self.load_attendance()
    
    def load_attendance(self):
        if os.path.exists(ATTENDANCE_FILE):
            try:
                with open(ATTENDANCE_FILE, 'r') as f:
                    self.attendance_records = json.load(f)
            except Exception as e:
                print(f"Error loading attendance: {e}")
                self.attendance_records = []
    
    def save_attendance(self):
        try:
            with open(ATTENDANCE_FILE, 'w') as f:
                json.dump(self.attendance_records, f, indent=4)
        except Exception as e:
            print(f"Error saving attendance: {e}")
    
    def load_embeddings(self):
        if os.path.exists(EMBEDDING_PATH):
            try:
                with open(EMBEDDING_PATH, 'rb') as f:
                    return pickle.load(f)
            except Exception as e:
                print(f"Error loading embeddings: {e}")
        return {}
    
    def save_embeddings(self, data):
        try:
            with open(EMBEDDING_PATH, 'wb') as f:
                pickle.dump(data, f)
        except Exception as e:
            print(f"Error saving embeddings: {e}")
    
    def mark_present(self, name):
        now = datetime.now()
        time_str = now.strftime("%Y-%m-%d %H:%M:%S")
        
        if name not in self.current_session:
            self.current_session[name] = time_str
            return True
        return False
    
    def check_attendance_time(self):
        if not self.attendance_active or self.last_attendance_time is None:
            return False
            
        now = datetime.now()
        if (now - self.last_attendance_time) >= timedelta(minutes=ATTENDANCE_INTERVAL):
            self.finalize_attendance()
            self.last_attendance_time = None
            self.attendance_active = False
            return True
        return False
    
    def start_attendance_session(self):
        self.last_attendance_time = datetime.now()
        self.attendance_active = True
        self.current_session = {}
    
    def finalize_attendance(self):
        if not self.current_session:
            return
            
        db = self.load_embeddings()
        all_names = set(db.keys())
        present_names = set(self.current_session.keys())
        absent_names = all_names - present_names
        
        session_data = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "present": {name: self.current_session[name] for name in present_names},
            "absent": list(absent_names)
        }
        
        self.attendance_records.append(session_data)
        self.save_attendance()
        self.current_session = {}
        return session_data
    
    def identify_face(self, embedding, threshold=0.7):
        db = self.load_embeddings()
        if not db:
            return None
            
        identity = None
        min_dist = float('inf')
        
        for name, known_emb in db.items():
            # Calculate Euclidean distance between embeddings
            dist = torch.norm(known_emb - embedding, p=2).item()
            
            if dist < min_dist and dist < threshold:
                min_dist = dist
                identity = name
                
        return identity

class FaceAttendanceApp:
    def __init__(self, master):
        self.master = master
        self.attendance_system = AttendanceSystem()
        self.cap = None
        
        # Window configuration
        master.title("Smart Attendance System")
        master.geometry("1000x700")
        master.configure(bg='#f5f5f5')
        
        # Custom style
        self.style = ttk.Style()
        self.style.theme_use('clam')
        
        # Color scheme
        self.colors = {
            'primary': '#4e73df',
            'success': '#1cc88a',
            'info': '#36b9cc',
            'warning': '#f6c23e',
            'danger': '#e74a3b',
            'light': '#f8f9fc',
            'dark': '#5a5c69'
        }
        
        # Configure styles
        self.configure_styles()
        
        # Header frame
        header_frame = ttk.Frame(master, style='Header.TFrame')
        header_frame.pack(fill='x', padx=10, pady=10)
        
        ttk.Label(header_frame, text="Smart Attendance System", 
                 style='Header.TLabel').pack(pady=15)
        
        # Main content frame
        main_frame = ttk.Frame(master)
        main_frame.pack(fill='both', expand=True, padx=20, pady=10)
        
        # Left panel - Controls
        control_frame = ttk.Frame(main_frame, style='Card.TFrame')
        control_frame.pack(side='left', fill='y', padx=(0, 10))
        
        ttk.Label(control_frame, text="System Controls", 
                 style='CardHeader.TLabel').pack(pady=10)        
        control_buttons = [
            ("Add New Student", self.add_face_gui, 'primary'),
            ("Upload Video", self.upload_video_for_registration, 'info'),
            ("Start Attendance", self.recognize_faces, 'success'),
            ("Manage Students", self.manage_users, 'info'),
            ("View Attendance", self.view_attendance, 'warning'),
            ("Edit Attendance", self.edit_attendance, 'primary')
        ]
        
        for text, command, color in control_buttons:
            btn = ttk.Button(control_frame, text=text, command=command,
                            style=f'{color}.TButton')
            btn.pack(fill='x', padx=10, pady=5)
        
        # Right panel - Status
        status_frame = ttk.Frame(main_frame, style='Card.TFrame')
        status_frame.pack(side='right', fill='both', expand=True)
        
        ttk.Label(status_frame, text="Attendance Status", 
                 style='CardHeader.TLabel').pack(pady=10)
        
        self.status_text = scrolledtext.ScrolledText(status_frame, 
                                                   height=20,
                                                   font=('Helvetica', 10),
                                                   wrap=tk.WORD)
        self.status_text.pack(fill='both', expand=True, padx=10, pady=5)
        
        # Footer
        footer_frame = ttk.Frame(master, style='Footer.TFrame')
        footer_frame.pack(fill='x', padx=10, pady=10)
        
        self.time_label = ttk.Label(footer_frame, 
                 text="Attendance not started",
                 style='Footer.TLabel')
        self.time_label.pack()
    
    def configure_styles(self):
        self.style.configure('Header.TFrame', background=self.colors['primary'])
        self.style.configure('Header.TLabel', 
                           background=self.colors['primary'],
                           foreground='white',
                           font=('Helvetica', 18, 'bold'))
        
        self.style.configure('Card.TFrame', 
                           background='white',
                           relief='groove',
                           borderwidth=2)
        
        self.style.configure('CardHeader.TLabel',
                           font=('Helvetica', 12, 'bold'),
                           background='white',
                           foreground=self.colors['dark'])
        
        self.style.configure('Footer.TFrame',
                           background=self.colors['light'])
        
        self.style.configure('Footer.TLabel',
                           background=self.colors['light'],
                           font=('Helvetica', 10))
        
        for color_name, color_code in self.colors.items():
            if color_name in ['primary', 'success', 'info', 'warning', 'danger']:
                self.style.configure(f'{color_name}.TButton',
                                    foreground='white',
                                    background=color_code,
                                    font=('Helvetica', 10, 'bold'),
                                    padding=8)
                
                self.style.map(f'{color_name}.TButton',
                              background=[('active', color_code), ('disabled', '#dddddd')])
    
    def update_status(self, message):
        self.status_text.insert(tk.END, message + "\n")
        self.status_text.see(tk.END)
        self.status_text.update()
    
    def add_face_gui(self):
        def start_video_capture():
            name = name_var.get().strip()
            if not name:
                messagebox.showerror("Error", "Please enter a valid name")
                return
            
            top.destroy()
            self.capture_video_embeddings(name)
        
        top = Toplevel(self.master)
        top.title("Register New Student")
        top.geometry("350x200")
        top.configure(bg='white')
        
        ttk.Label(top, text="Enter Student Name:", 
                 style='CardHeader.TLabel').pack(pady=10)
        
        name_var = StringVar()
        name_entry = ttk.Entry(top, textvariable=name_var, font=('Helvetica', 12))
        name_entry.pack(pady=5, padx=20, fill='x')
        
        btn_frame = ttk.Frame(top)
        btn_frame.pack(pady=10)
        
        ttk.Button(btn_frame, text="Record Video", 
                  command=start_video_capture, style='primary.TButton').pack(side='left', padx=5)
        ttk.Button(btn_frame, text="Cancel", 
                  command=top.destroy, style='danger.TButton').pack(side='left', padx=5)
    
    def upload_video_for_registration(self):
        top = Toplevel(self.master)
        top.title("Upload Video for Registration")
        top.geometry("400x200")
        
        ttk.Label(top, text="Enter Student Name:").pack(pady=10)
        name_var = StringVar()
        name_entry = ttk.Entry(top, textvariable=name_var)
        name_entry.pack(pady=5, padx=20, fill='x')
        
        def upload_and_process():
            name = name_var.get().strip()
            if not name:
                messagebox.showerror("Error", "Please enter a valid name")
                return
                
            filepath = filedialog.askopenfilename(
                title="Select Video File",
                filetypes=[("Video Files", "*.mp4 *.avi *.mov"), ("All Files", "*.*")]
            )
            
            if not filepath:
                return
                
            top.destroy()
            self.process_uploaded_video(filepath, name)
            
        ttk.Button(top, text="Upload Video", command=upload_and_process).pack(pady=10)
        ttk.Button(top, text="Cancel", command=top.destroy).pack(pady=5)
    
    def process_uploaded_video(self, video_path, name):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            messagebox.showerror("Error", "Could not open video file")
            return
            
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        progress_window = Toplevel(self.master)
        progress_window.title("Processing Video")
        progress_window.geometry("400x150")
        
        progress_label = ttk.Label(progress_window, text=f"Processing {os.path.basename(video_path)}")
        progress_label.pack(pady=10)
        
        progress_var = tk.DoubleVar()
        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, maximum=frame_count)
        progress_bar.pack(pady=10)
        
        embeddings = []
        frame_number = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            frame_number += 1
            progress_var.set(frame_number)
            progress_window.update()
            
            # Process frame
            faces = self.attendance_system.recognizer.detect_faces(frame)
            
            if faces:
                for face in faces:
                    if face['embedding'] is not None:
                        embeddings.append(face['embedding'])
            
            if len(embeddings) >= MIN_EMBEDDINGS:
                break
        
        cap.release()
        progress_window.destroy()
        
        if len(embeddings) >= MIN_EMBEDDINGS:
            # Average the embeddings
            avg_embedding = torch.mean(torch.stack(embeddings), dim=0)
            
            # Save to database
            db = self.attendance_system.load_embeddings()
            db[name] = avg_embedding
            self.attendance_system.save_embeddings(db)
            
            # Save a representative frame
            img_path = os.path.join("faces", f"{name}.jpg")
            cv2.imwrite(img_path, frame)
            
            messagebox.showinfo("Success", 
                              f"{name} registered successfully with {len(embeddings)} embeddings!")
        else:
            messagebox.showerror("Error", 
                               f"Could only capture {len(embeddings)} embeddings. Please try with a better quality video.")
    
    def capture_video_embeddings(self, name):
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Could not open video device")
            return
            
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        video_path = os.path.join("faces", f"{name}.avi")
        out = cv2.VideoWriter(video_path, fourcc, 20.0, (640, 480))
        
        embeddings = []
        start_time = time.time()
        capture_complete = False
        
        progress_window = Toplevel(self.master)
        progress_window.title("Recording Video")
        progress_window.geometry("400x150")
        
        progress_label = ttk.Label(progress_window, text="Please move your head slowly in different directions")
        progress_label.pack(pady=10)
        
        progress_var = tk.DoubleVar()
        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, 
                                      maximum=VIDEO_LENGTH, length=350)
        progress_bar.pack(pady=10)
        
        time_label = ttk.Label(progress_window, text=f"Time remaining: {VIDEO_LENGTH} seconds")
        time_label.pack()
        
        def update_progress():
            while not capture_complete:
                elapsed = time.time() - start_time
                remaining = max(0, VIDEO_LENGTH - elapsed)
                progress_var.set(elapsed)
                time_label.config(text=f"Time remaining: {int(remaining)} seconds")
                if elapsed >= VIDEO_LENGTH:
                    break
                time.sleep(0.1)
        
        progress_thread = Thread(target=update_progress)
        progress_thread.start()
        
        while (time.time() - start_time) < VIDEO_LENGTH:
            ret, frame = self.cap.read()
            if not ret:
                continue
            
            out.write(frame)
            
            # Process frame
            faces = self.attendance_system.recognizer.detect_faces(frame)
            
            if faces:
                for face in faces:
                    if face['embedding'] is not None:
                        embeddings.append(face['embedding'])
            
            cv2.putText(frame, "Move your head slowly in different directions", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Embeddings captured: {len(embeddings)}/{MIN_EMBEDDINGS}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Time remaining: {int(VIDEO_LENGTH - (time.time() - start_time))}s", 
                        (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.imshow("Recording Video - Move Your Head", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            if len(embeddings) >= MIN_EMBEDDINGS:
                break
        
        capture_complete = True
        self.cap.release()
        out.release()
        cv2.destroyAllWindows()
        progress_window.destroy()
        
        if len(embeddings) >= MIN_EMBEDDINGS:
            avg_embedding = torch.mean(torch.stack(embeddings), dim=0)
            
            db = self.attendance_system.load_embeddings()
            db[name] = avg_embedding
            self.attendance_system.save_embeddings(db)
            
            img_path = os.path.join("faces", f"{name}.jpg")
            cv2.imwrite(img_path, frame)
            
            messagebox.showinfo("Success", 
                              f"{name} registered successfully with {len(embeddings)} embeddings!")
        else:
            os.remove(video_path)
            messagebox.showerror("Error", 
                               f"Could only capture {len(embeddings)} embeddings. Please try again.")
    
    def recognize_faces(self):
        self.attendance_system.start_attendance_session()
        self.update_status("\nAttendance session started...")
        self.update_timer()
        
        db = self.attendance_system.load_embeddings()
        if not db:
            messagebox.showinfo("Info", "No registered students found")
            return
        
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Could not open video device")
            return
        
        self.update_status("Starting face recognition...")
        
        while True:
            if not self.attendance_system.attendance_active:
                self.update_status("Attendance period completed. Session closed.")
                break
                
            ret, frame = self.cap.read()
            if not ret:
                self.update_status("Error reading frame from camera")
                break
            
            if self.attendance_system.check_attendance_time():
                session_data = self.attendance_system.finalize_attendance()
                self.update_status(f"Attendance marked at {datetime.now().strftime('%H:%M:%S')}")
                self.update_status(f"Present: {len(session_data['present'])}")
                self.update_status(f"Absent: {len(session_data['absent'])}")
                break
            
            # Detect and track faces
            faces = self.attendance_system.recognizer.detect_faces(frame)
            tracked_faces = self.attendance_system.tracker.update(faces)
            
            for face in tracked_faces:
                x1, y1, x2, y2 = map(int, face['bbox'])
                
                if face.get('embedding') is not None:
                    identity = self.attendance_system.identify_face(face['embedding'])
                    
                    if identity:
                        is_new = self.attendance_system.mark_present(identity)
                        
                        if is_new:
                            self.update_status(f"{identity} marked present at {datetime.now().strftime('%H:%M:%S')}")
                            label = f"{identity} (Present)"
                            color = (0, 120, 255)  # Orange
                        else:
                            label = f"{identity} (Already marked)"
                            color = (0, 180, 0)  # Dark green
                    else:
                        label = "Unknown"
                        color = (0, 0, 255)  # Red
                    
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, label, (x1, y1-10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            if self.attendance_system.last_attendance_time:
                time_left = (self.attendance_system.last_attendance_time + 
                            timedelta(minutes=ATTENDANCE_INTERVAL) - datetime.now())
                mins, secs = divmod(max(0, time_left.seconds), 60)
                time_text = f"Time left: {mins:02d}:{secs:02d}"
                
                cv2.rectangle(frame, (5, 5), (250, 40), (0, 0, 0), -1)
                cv2.putText(frame, time_text, (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.imshow("Face Recognition - Press Q to Quit", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.attendance_system.attendance_active = False
                break
        
        self.cap.release()
        cv2.destroyAllWindows()
    
    def update_timer(self):
        if self.attendance_system.attendance_active and self.attendance_system.last_attendance_time:
            time_left = (self.attendance_system.last_attendance_time + 
                        timedelta(minutes=ATTENDANCE_INTERVAL) - datetime.now())
            if time_left.total_seconds() > 0:
                mins, secs = divmod(time_left.seconds, 60)
                self.time_label.config(text=f"Time left: {mins:02d}:{secs:02d}")
                self.master.after(1000, self.update_timer)
            else:
                self.time_label.config(text="Attendance session completed")
        else:
            self.time_label.config(text="Attendance not started")
    
    def manage_users(self):
        top = Toplevel(self.master)
        top.title("Student Management")
        top.geometry("900x600")
        top.configure(bg='white')
        
        main_frame = ttk.Frame(top)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Registered Students", 
                 style='Header.TLabel').pack(pady=10)
        
        canvas = tk.Canvas(main_frame, bg='white')
        scrollbar = ttk.Scrollbar(main_frame, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        db = self.attendance_system.load_embeddings()
        users = list(db.keys())
        
        if not users:
            ttk.Label(scrollable_frame, text="No registered students").pack()
            return
        
        row_frame = None
        for i, user in enumerate(users):
            if i % 4 == 0:
                row_frame = ttk.Frame(scrollable_frame)
                row_frame.pack(fill='x', pady=5)
            
            card_frame = ttk.Frame(row_frame, style='Card.TFrame')
            card_frame.pack(side='left', padx=10, pady=5)
            
            img_path = os.path.join("faces", f"{user}.jpg")
            if os.path.exists(img_path):
                img = Image.open(img_path).resize((120, 120))
                photo = ImageTk.PhotoImage(img)
                lbl = ttk.Label(card_frame, image=photo)
                lbl.image = photo
                lbl.pack(pady=5)
            
            ttk.Label(card_frame, text=user, 
                     style='CardHeader.TLabel').pack()
            
            ttk.Button(card_frame, text="Delete", 
                      command=lambda u=user: self.delete_user(u, top),
                      style='danger.TButton').pack(pady=5)
    
    def delete_user(self, user, window):
        if not messagebox.askyesno("Confirm", f"Delete {user} permanently?"):
            return
        
        db = self.attendance_system.load_embeddings()
        if user in db:
            del db[user]
            self.attendance_system.save_embeddings(db)
            
            img_path = os.path.join("faces", f"{user}.jpg")
            if os.path.exists(img_path):
                os.remove(img_path)
            
            video_path = os.path.join("faces", f"{user}.avi")
            if os.path.exists(video_path):
                os.remove(video_path)
            
            messagebox.showinfo("Success", f"Student {user} deleted successfully")
            window.destroy()
            self.manage_users()
        else:
            messagebox.showerror("Error", "Student not found")
    
    def view_attendance(self):
        top = Toplevel(self.master)
        top.title("Attendance Records")
        top.geometry("1000x600")
        
        main_frame = ttk.Frame(top)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Attendance History", 
                 style='Header.TLabel').pack(pady=10)
        
        text_frame = ttk.Frame(main_frame)
        text_frame.pack(fill='both', expand=True)
        
        scrollbar = ttk.Scrollbar(text_frame)
        scrollbar.pack(side='right', fill='y')
        
        attendance_text = tk.Text(text_frame, wrap=tk.WORD, 
                                yscrollcommand=scrollbar.set,
                                font=('Courier New', 10))
        attendance_text.pack(fill='both', expand=True)
        
        scrollbar.config(command=attendance_text.yview)
        
        if not os.path.exists(ATTENDANCE_FILE):
            attendance_text.insert(tk.END, "No attendance records found")
            return
        
        with open(ATTENDANCE_FILE, 'r') as f:
            records = json.load(f)
        
        for record in records:
            timestamp = record['timestamp']
            present = record['present']
            absent = record['absent']
            
            attendance_text.insert(tk.END, f"\n=== Session: {timestamp} ===\n")
            attendance_text.insert(tk.END, f"Present ({len(present)}):\n")
            for name, time in present.items():
                attendance_text.insert(tk.END, f"  - {name} at {time}\n")
            
            attendance_text.insert(tk.END, f"\nAbsent ({len(absent)}):\n")
            for name in absent:
                attendance_text.insert(tk.END, f"  - {name}\n")
            
            attendance_text.insert(tk.END, "\n" + "="*40 + "\n")
        
        attendance_text.config(state='disabled')
    
    def edit_attendance(self):
        edit_window = Toplevel(self.master)
        edit_window.title("Edit Attendance")
        edit_window.geometry("800x600")
        
        main_frame = ttk.Frame(edit_window)
        main_frame.pack(fill='both', expand=True, padx=20, pady=20)
        
        ttk.Label(main_frame, text="Edit Attendance Records", 
                 style='Header.TLabel').pack(pady=10)
        
        date_frame = ttk.Frame(main_frame)
        date_frame.pack(fill='x', pady=10)
        
        ttk.Label(date_frame, text="Select Session:").pack(side='left')
        
        if not os.path.exists(ATTENDANCE_FILE):
            messagebox.showerror("Error", "No attendance records found")
            edit_window.destroy()
            return
        
        with open(ATTENDANCE_FILE, 'r') as f:
            records = json.load(f)
        
        session_dates = [r['timestamp'] for r in records]
        session_var = StringVar()
        session_dropdown = ttk.Combobox(date_frame, textvariable=session_var, 
                                       values=session_dates, state='readonly')
        session_dropdown.pack(side='left', padx=10)
        session_dropdown.current(0)
        
        student_frame = ttk.Frame(main_frame)
        student_frame.pack(fill='both', expand=True)
        
        student_status_vars = {}
        
        def load_session():
            selected_session = session_var.get()
            if not selected_session:
                return
            
            session_data = next((r for r in records if r['timestamp'] == selected_session), None)
            if not session_data:
                return
            
            for widget in student_frame.winfo_children():
                widget.destroy()
            
            db = self.attendance_system.load_embeddings()
            all_students = list(db.keys())
            
            for student in all_students:
                row_frame = ttk.Frame(student_frame)
                row_frame.pack(fill='x', pady=2)
                
                ttk.Label(row_frame, text=student, width=20).pack(side='left')
                
                status_var = StringVar()
                status_var.set('present' if student in session_data['present'] else 'absent')
                
                ttk.Radiobutton(row_frame, text="Present", variable=status_var, 
                               value='present').pack(side='left', padx=5)
                ttk.Radiobutton(row_frame, text="Absent", variable=status_var, 
                               value='absent').pack(side='left', padx=5)
                
                student_status_vars[student] = status_var
        
        ttk.Button(date_frame, text="Load Session", command=load_session, 
                  style='primary.TButton').pack(side='left')
        
        def save_changes():
            selected_session = session_var.get()
            if not selected_session:
                return
            
            session_index = next((i for i, r in enumerate(records) 
                                if r['timestamp'] == selected_session), None)
            if session_index is None:
                return
            
            present = {}
            absent = []
            
            for student, var in student_status_vars.items():
                if var.get() == 'present':
                    original_time = records[session_index]['present'].get(student, 
                        datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    present[student] = original_time
                else:
                    absent.append(student)
            
            records[session_index]['present'] = present
            records[session_index]['absent'] = absent
            
            with open(ATTENDANCE_FILE, 'w') as f:
                json.dump(records, f, indent=4)
            
            messagebox.showinfo